In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np


    

In [7]:
def pcafunction(x_data, y_data, num_componentes, name_column, name_index):

    # standarizar
    scaler_model = StandardScaler().fit(x_data)
    x = scaler_model.transform(x_data) 
 
    
    
    pca = PCA(n_components=num_componentes)
    principalComponents = pca.fit_transform(x)
    
    pca_df = pd.DataFrame(principalComponents, columns=[f'PC1_{name_column}', f'PC2_{name_column}'])
    pca_df['FECHA-HORA'] = y_data
    
    
    
    return pca,pca_df,scaler_model

In [8]:
def exogen_pca(dataframe, var_index):
   
    #variables
    dfvar = dataframe[['DEMANDA', 'MES', 'DIA', 'HORA','TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR', 'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB',
       'TMIN-HMO', 'TMIN-OBR', 'TMIN-LMO', 'TMIN-CUL', 'PREC_HMO_MM',
       'PREC_OBR_MM', 'PREC_LMO_MM', 'PREC_CUL_MM', 'LUNES_FESTIVO',
       'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO', '10_MAYO', '16_SEP',
       '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR', 'NAVIDAD_Y_NEW_YEAR',
       'POST-NAVIDAD_Y_NEW_YEAR']]
    
    dfvar['FECHA-HORA']= dfvar.index
    
    #variables para pca
    y = dataframe.loc[:,[var_index]].values
    
    #PCA CLIMATOLOGICAS
    x_climatologicas = dataframe.loc[:,'TMAX-CAB':'PREC_CUL_MM'].values 
    pca_clima,pca_df_clima, scaler_model_clima = pcafunction(x_climatologicas, y, 2, 'CLIMA', var_index)
    
    
    #UNION DE LOS 3 DATAFRAMES
    pca_df = pd.merge(pca_df_clima, dfvar, left_on=var_index, right_on='FECHA-HORA', how='left')
    del pca_df['FECHA-HORA']
    
    return pca_df,pca_clima,scaler_model_clima

In [9]:
def new_dataframe(my_array,y_data,name_column, name_index ):
    df = pd.DataFrame(my_array, columns=[f'PC1_{name_column}', f'PC2_{name_column}'])
    df['FECHA-HORA'] = y_data
    
    return df

In [10]:
def transform_data(dataframe, var_index,pca_clima,scaler_model_clima):
    
    #variables
    dfvar = dataframe[['DEMANDA', 'MES', 'DIA', 'HORA','LUNES_FESTIVO',
       'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO', '10_MAYO', '16_SEP',
       '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR', 'NAVIDAD_Y_NEW_YEAR',
       'POST-NAVIDAD_Y_NEW_YEAR']]
    dfvar['FECHA-HORA']= dfvar.index
    #variables y
    y = dfvar.index
    
    ##### PCA CLIMA #####
    #varibales de x
    x_clima = dataframe.loc[:,'TMAX-CAB':'PREC_CUL_MM'].values 
    #standarizacion  CLIMA
    x_df_clima = scaler_model_clima.transform(x_clima) 
    #trasnformacion
    clima = pca_clima.transform(x_df_clima)
    
    
    df_clima = new_dataframe(clima,y,'CLIMA', var_index )
    
    #UNION DE LOS 3 DATAFRAMES
    pca_df = pd.merge(df_clima, dfvar, left_on=var_index, right_on='FECHA-HORA', how='left')
    #del pca_df['FECHA-HORA']

    return pca_df
    